In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
cans = pd.read_csv('PittsburghSmartTrash.csv')
neighborhood = pd.read_csv('neighborhoods.csv')

In [14]:
fire_incident['neighborhood'].value_counts()

Carrick                      255
Squirrel Hill South          225
Central Business District    225
South Side Flats             219
Hazelwood                    213
                            ... 
St. Clair                     11
Mt. Oliver                    10
East Carnegie                 10
Regent Square                  7
Mount Oliver Borough           6
Name: neighborhood, Length: 91, dtype: int64

In [15]:
sortedarea = neighborhood.sort_values(by = "hood")
sortedareadf = sortedarea[['hood','acres']]
newsortedareadf = sortedareadf.reset_index(drop=True)
newsortedareadf

,hood,acres
0,Allegheny Center,134.703
1,Allegheny West,90.841
2,Allentown,189.608
3,Arlington,302.320
4,Arlington Heights,84.592
...,...,...
85,Upper Lawrenceville,260.349
86,West End,128.679
87,West Oakland,138.372
88,Westwood,445.009


In [16]:
new_df = fire_incident.sort_values(["neighborhood"], ascending=True)
fire_incident['neighborhood'].value_counts()

newdf =fire_incident['neighborhood'].value_counts().rename_axis('neighborhood').reset_index(name='frequency')
sorteddf = newdf.sort_values(["neighborhood"], ascending=True)
sortednew = sorteddf.reset_index(drop=True)
mergesorteddata = sortednew.merge(newsortedareadf, left_on = 'neighborhood', right_on = 'hood', suffixes=('frequency', 'acres'))
mergesorteddata.sort_values(['frequency'], ascending=True)
mergesorteddata['ratio'] = mergesorteddata['frequency'] / mergesorteddata['acres']
ratio = mergesorteddata.sort_values(["ratio"], ascending=True)
ratio

,neighborhood,frequency,hood,acres,ratio
37,Hays,18,Hays,1125.478,0.015993
26,East Carnegie,10,East Carnegie,277.895,0.035985
31,Fairywood,23,Fairywood,622.746,0.036933
54,New Homestead,20,New Homestead,510.728,0.039160
81,Swisshelm Park,15,Swisshelm Park,308.575,0.048611
...,...,...,...,...,...
25,East Allegheny,111,East Allegheny,151.705,0.731683
43,Knoxville,146,Knoxville,193.982,0.752647
49,Marshall-Shadeland,119,Marshall-Shadeland,148.209,0.802920
46,Lincoln-Lemington-Belmar,172,Lincoln-Lemington-Belmar,180.105,0.954998
